# Stats in R2lab

### Convenience

This cell is only here so that any change in the code gets reloaded.

In [ ]:
%load_ext autoreload
%autoreload 2

## Scope

The working assumption here is that the interesting data is

* the percentage of usage over a given period `[start .. end]`

* the total number of relevant accounts and slices

* also we might wish to pinpoint entries in the db that
  correspond to some operation conditions. For example,
  as of march 2018, we are interested in the disabled accounts
  attached to the r2lab site, because we suspect some people in
  this set have been trying to join but that was never acted upon...

## Changelog

* 2018 October; using this to prepare the stats exposed in the FIT meeting on Oct. 16

* 2018 March; this is a rebuild - see `stats-old.py` - of a previously, rather *ad hoc* script. 
  The present version will be OK for mostly 2017 and later, as we ignore the old data stored in json files.

* 2017 November; at that time - again, see `stats-old.py` - we used 2 different sources of data, presumably because of the migration from the omf/rest API to myplc.

## Prerequisite

<div style="border: 3px dotted; text-align: center; background-color:red"><b>IMPORTANT !!</b> Walid you need to read this !</div>

* we need to have the complete list of slices, but PLCAPI won't let us access to slices that are deleted - so essentially the ones that have expired;

* so in order to compensate for that, you need to run the script `gather-slices.py` on `r2labapi.inria.fr` and then retrieve the corresponding output here - typically a file named `SLICES-2018-03-23.json`

```
[root@r2labapi ~]# cd r2lab-misc/usage-statistics/
[root@r2labapi usage-statistics]# git pull
Updating 17c7136..d892b36
Fast-forward
 usage-statistics/gather-slices.py    | 73 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 usage-statistics/stats-2018-03.ipynb | 39 +++++++++++++++++++++++++++++++++------
 2 files changed, 106 insertions(+), 6 deletions(-)
 create mode 100755 usage-statistics/gather-slices.py
[root@r2labapi usage-statistics]# ./gather-slices.py
(Over)wrote SLICES-2018-03-23.json
```

Then make sure to retrieve that file locally and define its name here:

```
tparment ~/git/r2lab-misc/usage-statistics (master=) $ rsync -ai $(plr r2labapi):r2lab-misc/usage-statistics/SLICES\* .
>f..t.... SLICES-2018-03-23.json
```

In [ ]:
slices_filename = "SLICES-2018-10-12.json"

In [ ]:
import json

with open(slices_filename) as feed:
    all_slices = json.loads(feed.read())
    print("SLICES files loaded OK")

# Proxying / password to the API

You need to know the password of the PLCAPI admin account on `r2labapi.inria.fr`

In [ ]:
# only prompt once so the notebook can be re-run often
import getpass
account = "root@r2lab.inria.fr"
try:
    if password:
        print("We know the password")
    else:
        raise ValueError
except:
    password = getpass.getpass(f"Enter password for {account} : ")

In [ ]:
# let's check that
auth = {'AuthMethod' : 'password',
        'Username'   : account,
        'AuthString' : password}

import xmlrpc.client
url = "https://r2labapi.inria.fr:443/PLCAPI/"

In [ ]:
proxy = xmlrpc.client.ServerProxy(url)
try:
    print("Authorization OK" if proxy.AuthCheck(auth)==1 else "KO")
except Exception as e:
    print(f"OOPS, something wrong with {type(e)} - {e}")

*********

## Enter your period of interest

Second argument to `input_date` is the default answer

In [ ]:
from timeutils import show_period, human_readable

In [ ]:
show_period();

# Fetching user accounts

In [ ]:
all_accounts = proxy.GetPersons(auth)

print(f"We have {len(all_accounts)} accounts in the DB")

In [ ]:
# accessory

# This will add a `login_base` field to each user account
# to identify the attached institution:

all_sites = proxy.GetSites(auth)
site_hash = { site['site_id'] : site for site in all_sites}
for account in all_accounts:
    site_ids = account['site_ids']
    if not site_ids:
        account['login_base'] = '(none)'
    if len(site_ids) == 1:
        account['login_base'] = site_hash[account['site_ids'][0]]['login_base']
    else:
        account['login_base'] = '(' + ",".join(site_hash[site_id]['login_base'] for site_id in site_ids) + ')'

# Classifications - import data from excel

Let's load Walid's paper on the accounts; this is to be able to classify usages and accounts into several categories.

In [ ]:
import pandas as pd

Originally this is an excel file `accounts-annotations.xls`. We need an extra library to be able to read `xlsx` files directly:

    pip3 install xlrd

In [ ]:
# read excel file
df = pd.read_excel('accounts-annotations.xlsx', encoding='cp1252')

In [ ]:
# our main index is the 'Mail' column
df = df.set_index('email')

In [ ]:
# visual check on a small sample
df.iloc[3:6]

### Annotate `all_accounts`

In [ ]:
for account in all_accounts:
    if not account['enabled']:
        # print(f"ignoring disabled account {account['email']}")
        continue
    try:
        email = account['email']
        excel_row = df.loc[email]
        # academia or industry
        family = excel_row['family']
        if family in ('academia', 'industry'):
            account['family'] = family
        else:
            print(f"Unknown family for {email} ! (in {account['login_base']})")
        if excel_row['diana'] == 'yes':
            account['scope'] = 'diana'
        elif excel_row['fit'] == 'yes':
            account['scope'] = 'fit'
        elif excel_row['others'] == 'yes':
            account['scope'] = 'others'
        else:
            print(f"Unknown scope for {email} ! (in {account['login_base']})")
    except Exception as e:
        print(f'OOPS with email={email} - {type(e)} - {e}')


If the above cell has not printed any warning, we have all active user accounts tagged with 
* `family` as either `academia` or industry
* `scope` as `diana`, `fit` or `others`

# Stats on user accounts

In [ ]:
def show_accounts(accounts):
    accounts.sort(key = lambda person: person['date_created'])
    for i, account in enumerate(accounts, 1):
        default = 'n/a' if account['enabled'] else '--'
        print(f"[{i:02d}] "
              f" {'OK' if account['enabled'] else 'KO'}"
              f" {human_readable(account['date_created'])}"
              f" {account['login_base']:22s}",
              f" {account.get('family', default):8s}",
              f" {account.get('scope', default):8s}",
              f" {account['email']}")

### narrowing on the selected period

In [ ]:
ifrom, iuntil = show_period()
selected_accounts = [
    account for account in all_accounts 
    if account['date_created'] >= ifrom and
       account['date_created'] <= iuntil
]

In [ ]:
# show_accounts(selected_accounts)
print(f"a total of {len(selected_accounts)} accounts were created over the selected period")

### focusing on enabled accounts (What really matters)

In [ ]:
enabled_accounts_in_selected_period = [
    account for account in selected_accounts if account['enabled']
]

In [ ]:
print(f"New enabled accounts in the selected period = {len(enabled_accounts_in_selected_period)}")

In [ ]:
show_accounts(enabled_accounts_in_selected_period)

### By scope : diana / fit / others

In [ ]:
for scope in ['diana', 'fit', 'others']:
    scope_accounts = [account for account in enabled_accounts_in_selected_period
                       if account['scope'] == scope]
    print(f"in scope {scope}, {len(scope_accounts)} new enabled accounts")

### By family : academia / industry

In [ ]:
for family in ['academia', 'industry']:
    family_accounts = [account for account in enabled_accounts_in_selected_period
                       if account['family'] == family]
    print(f"in family {family}, {len(family_accounts)} new enabled accounts")

# Classifying slices

In [ ]:
# sort in expiration order    
all_slices.sort(key = lambda slice: slice['expires'])
    
print(f"Found {len(all_slices)} slices")    

In [ ]:
def show_slices(slices):
    for i, slice in enumerate(slices, 1):
        print(f"{i:02d} "
            f" created {human_readable(slice['created'])}"
            f" expires {human_readable(slice['expires'])}"
            f" {slice['name']}"
         )

In [ ]:
show_slices(all_slices)

### Ignoring admin slices

In [ ]:
admin_slices = ['auto_', 'nightly', 'maintenance' ]

def relevant(slice_or_lease):
    return not any(admin in slice_or_lease['name'] for admin in admin_slices)

### classifying slices

In [ ]:
# hash accounts per person_id
accounts_hash = { account['person_id'] : account for account in all_accounts}

for slice in all_slices:
    if not relevant(slice):
        continue
    person_ids = slice['person_ids']
    slice['families'] = [accounts_hash[person_id].get('family', '???')
                         for person_id in slice['person_ids']]

In [ ]:
for slice in all_slices:
    if not relevant(slice):
        continue
    print(f"slice {slice['name']} has {len(slice['person_ids'])} people => {slice['families']}")

Long story short, at this point we can safely consider that 100% of our usage is academy.

# Fetching leases

In [ ]:
# fetch leases for that period
selected_leases = proxy.GetLeases(
    auth,
    {'>t_from' : ifrom, '<t_from' : iuntil}
)

# Sort then in ascending order
selected_leases.sort(key=lambda lease: lease['t_from'])

print(f"there have been {len(selected_leases)} reservations made during the period")

### A glimpse

In [ ]:
def lease_line(lease):
    return f"{lease['name']:25s} {human_readable(lease['t_from'])} -> {human_readable(lease['t_until'])}"

def glimpse(leases, size=5):
    for lease in leases[:size]:
        print(lease_line(lease))
    print("...")
    for lease in leases[-size:]:
        print(lease_line(lease))

In [ ]:
glimpse(selected_leases)

# Usage ratio

##### raw ratio *vs* opening hours

The raw ratio is obtained by comparing the amount of time reserved with the total amount of time available.

Assuming that opening hours would be mon-fri from 09:00 to 19:00

In [ ]:
# this is a constant

open_correction = (5 * 10) / (7 * 24)
print(f"Opening hours are {open_correction:.2%} of total hours")

##### user *vs* admin

We try to classify the various slices in 2 families whether they are for management/operations purposes, or used for actual experimentation.

In [ ]:
total_duration = iuntil - ifrom

In [ ]:
def show_usage_ratio(leases, total_duration, message):

    def duration(lease):
        return lease['t_until'] - lease['t_from']

    reserved_duration = sum(duration(lease) for lease in leases)
    print(f"Total time reserved: {reserved_duration} / {total_duration:1.0f} s")
    print(f"                i.e: {reserved_duration/3600:.2f} / {total_duration/3600:.2f}    hours")
    print(f"                i.e: {reserved_duration/(24*3600):.2f} / {total_duration/(24*3600):.2f}       days")
    
    raw_ratio = reserved_duration / total_duration
    print(f"{message}: raw_ratio is {raw_ratio:.2%}")
    
    open_ratio = raw_ratio / open_correction
    print(f"{message}: open_ratio is {open_ratio:.2%}")

In [ ]:
show_usage_ratio(selected_leases, total_duration, "ALL LEASES")

### Usage ratio - filtered

We discard slices whose name contains any of the following

In [ ]:
filtered_leases = [lease for lease in selected_leases if relevant(lease)]

In [ ]:
show_usage_ratio(filtered_leases, total_duration, "USER LEASES")

### validated *vs* non-validated accounts

Still on the selected period, show the ones that were enabled or not

In [ ]:
enabled_selected_accounts = [ account for account in selected_accounts if account['enabled']]
disabled_selected_accounts = [ account for account in selected_accounts if not account['enabled']]

In [ ]:
show_accounts(enabled_selected_accounts)

In [ ]:
show_accounts(disabled_selected_accounts)

*****
*****
*****

*Discarded* section about specific assessments made in March 2018 - see in the git history